# T5 on TPU 💥🚀

In [0]:
!wget http://ftp.acc.umu.se/mirror/wikimedia.org/dumps/viwiki/20200520/viwiki-20200520-pages-articles-multistream.xml.bz2

--2020-06-12 12:20:56--  http://ftp.acc.umu.se/mirror/wikimedia.org/dumps/viwiki/20200520/viwiki-20200520-pages-articles-multistream.xml.bz2
Resolving ftp.acc.umu.se (ftp.acc.umu.se)... 194.71.11.165, 194.71.11.173, 2001:6b0:19::165, ...
Connecting to ftp.acc.umu.se (ftp.acc.umu.se)|194.71.11.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 778935242 (743M) [application/x-bzip2]
Saving to: ‘viwiki-20200520-pages-articles-multistream.xml.bz2’

viwiki-20200520-pag 100%[===================>] 742.85M  22.6MB/s    in 35s     

2020-06-12 12:21:31 (21.5 MB/s) - ‘viwiki-20200520-pages-articles-multistream.xml.bz2’ saved [778935242/778935242]



In [0]:
!git clone https://github.com/attardi/wikiextractor.git

Cloning into 'wikiextractor'...
remote: Enumerating objects: 613, done.
remote: Total 613 (delta 0), reused 0 (delta 0), pack-reused 613
Receiving objects: 100% (613/613), 1.24 MiB | 9.14 MiB/s, done.
Resolving deltas: 100% (352/352), done.


In [0]:
!python wikiextractor/WikiExtractor.py --output /corpus --bytes 512M --compress --json --links --namespaces 0 --no_templates --min_text_length 16 --processes 20 viwiki-20200520-pages-articles-multistream.xml.bz2

Streaming output truncated to the last 5000 lines.
INFO: 1123068	Aculepeira escazu
INFO: 1123073	Aculepeira matsudae
INFO: 1123072	Aculepeira machu
INFO: 1123074	Aculepeira packardi
INFO: 1123076	Aculepeira talishia
INFO: 1123075	Aculepeira taibaishanensis
INFO: 1123077	Aculepeira travassosi
INFO: 1123079	Aculepeira vittata
INFO: 1123078	Aculepeira visite
INFO: 1123080	Acusilas africanus
INFO: 1123081	Acusilas callidus
INFO: 1123082	Acusilas coccineus
INFO: 1123083	Acusilas dahoneus
INFO: 1123084	Acusilas lepidus
INFO: 1123085	Acusilas malaccensis
INFO: 1123086	Acusilas spiralis
INFO: 1123087	Acusilas vei
INFO: 1123089	Aerea alticephala
INFO: 1123088	Acusilas vilei
INFO: 1123090	Aerea magnifica
INFO: 1123091	Aethriscus olivaceus
INFO: 1123093	Aethrodiscus transversalis
INFO: 1123092	Aethriscus pani
INFO: 1123094	Aetrocantha falkensteini
INFO: 1123095	Afracantha camerunensis
INFO: 1123096	Agalenatea liriope
INFO: 1123097	Alenatea fuscocolorata
INFO: 1123098	Alenatea touxie
INFO: 1123099

In [1]:
!git clone https://github.com/manhlab/bert-vietnamese.git

Cloning into 'bert-vietnamese'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 398 (delta 245), reused 325 (delta 185), pack-reused 0
Receiving objects: 100% (398/398), 293.21 KiB | 3.86 MiB/s, done.
Resolving deltas: 100% (245/245), done.


In [2]:
!sudo bash bert-vietnamese/install.sh

     |████████████████████████████████| 7.1MB 2.7MB/s 
     |████████████████████████████████| 1.5MB 68.2MB/s 
     |████████████████████████████████| 460kB 57.4MB/s 
     |████████████████████████████████| 757kB 54.9MB/s 
     |████████████████████████████████| 5.4MB 54.6MB/s 
     |████████████████████████████████| 286kB 55.1MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449906 sha256=836c1ae7d82649a4866c964189151e056930a72c137080711f3775355efdf6ff
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for clint: filename=clint-0.5.1-cp36-none-any.whl size=34449 sha256=59d8ea89a6845712de2b2d6bafe4ae559e51dccf0c8381b780e73bf3ced5997c
  Stored in directory: /root/.cache/pip/wheels/4f/e9/45/223565e5b1a4b09e12c6de6f8ba7c2c0e9127dec17cf830f83
  Created wheel for args: filename=args-0.1.0-cp36-none-any.whl size=3320 sha256=cc7011fbea66646b2509c2bac858408dd6db106b87793f224dc1d40173b02aab
  Store

In [0]:
!seq -f %02g 0 2|xargs -L 1 -I {} -P 9 python bert-vietnamese/make_corpus.py --input_file /corpus/AA/wiki_{}.bz2 --output_file /corpus/corpus.txt.{} --mecab_dict_path /path/to/neologd/dict/dir/

In [0]:
!head /corpus/corpus.txt.01

In [0]:
!mkdir /base
!touch /base/vocab.txt

In [0]:
!python bert-vietnamese/build_vocab.py --input_file "/corpus/corpus.txt.*" --output_file "/base/vocab.txt" --subword_type bpe --vocab_size 32000

In [0]:
!seq -f %02g 0 8|xargs -L 1 -I {} -P 1 python bert-vietnamese/create_pretraining_data.py --input_file /corpus/corpus.txt.{} --output_file /base/pretraining-data.tf_record.{} --do_whole_word_mask True --vocab_file /base/vocab.txt --subword_type bpe --max_seq_length 512 --max_predictions_per_seq 80 --masked_lm_prob 0.15

In [1]:
!seq -f %02g 0 8|xargs -L 1 -I {} -P 1 python bert-vietnamese/create_pretraining_data.py --input_file /corpus/corpus.txt.01 --output_file /base/pretraining-data.tf_record.01 --do_whole_word_mask True --vocab_file /base/vocab.txt --subword_type bpe --max_seq_length 512 --max_predictions_per_seq 80 --masked_lm_prob 0.15

Traceback (most recent call last):
  File "bert-vietnamese/create_pretraining_data.py", line 23, in <module>
    import tokenization
  File "/content/bert-vietnamese/tokenization.py", line 24, in <module>
    from transformers import BertTokenizer, WordpieceTokenizer
  File "/usr/local/lib/python3.6/dist-packages/transformers/__init__.py", line 20, in <module>
    from .file_utils import (TRANSFORMERS_CACHE, PYTORCH_TRANSFORMERS_CACHE, PYTORCH_PRETRAINED_BERT_CACHE,
  File "/usr/local/lib/python3.6/dist-packages/transformers/file_utils.py", line 24, in <module>
    from tqdm.auto import tqdm
  File "/usr/local/lib/python3.6/dist-packages/tqdm/__init__.py", line 1, in <module>
    from .std import tqdm, trange
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 12, in <module>
    from .utils import _supports_unicode, _environ_cols_wrapper, _range, _unich, \
  File "/usr/local/lib/python3.6/dist-packages/tqdm/utils.py", line 6, in <module>
    CUR_OS = _curos()
  File "/us

In [0]:
from google.colab import auth
auth.authenticate_user()

In [9]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  24185      0 --:--:-- --:--:-- --:--:-- 24185
OK
43 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 43 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...


In [10]:
!mkdir gs
!gcsfuse bertvietnamese gs

Using mount point: /content/gs
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [0]:
#!mv /base/pretraining-data.tf_record.* gs/tpuestimator-lstm/

In [3]:
import json
import os
import pprint
import re
import time
import tensorflow as tf


use_tpu = True #@param {type:"boolean"}
bucket = 'bertvietnamese' #@param {type:"string"}

assert bucket, 'Must specify an existing GCS bucket name'
print('Using bucket: {}'.format(bucket))

if use_tpu:

    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

MODEL_DIR = 'gs://{}/{}'.format(bucket, time.strftime('tpuestimator-lstm/%Y-%m-%d-%H-%M-%S'))
print('Using model dir: {}'.format(MODEL_DIR))

from google.colab import auth
auth.authenticate_user()

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
  
  # Upload credentials to TPU.
  with tf.Session(TF_MASTER) as sess:    
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  pprint.pprint(session.list_devices())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Using bucket: bertvietnamese
Using model dir: gs://bertvietnamese/tpuestimator-lstm/2020-06-14-06-35-09


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12469658639912646238),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3570114989057619173),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 2771035311076966166),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 17975435158694307510),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14721513887843458911),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 1665563704045150528)

In [0]:
!python3 bert-vietnamese/run_pretraining.py \
--input_file="gs://bertvietnamese/tpuestimator-lstm/pretraining-data.tf_record.*" \
--output_dir="gs://bertvietnamese/tpuestimator-lstm/2020-06-13-14-59-11/" \
--bert_config_file=bert-vietnamese/bert_base_32k_config.json \
--max_seq_length=512 \
--max_predictions_per_seq=80 \
--do_train=True \
--train_batch_size=128 \
--num_train_steps=100000 \
--learning_rate=1e-4 \
--save_checkpoints_steps=10000 \
--keep_checkpoint_max=10 \
--use_tpu=True \
--tpu_name=$TPU_NAME \
--num_tpu_cores=8 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 